<a href="https://colab.research.google.com/github/naxatra2/Movie-recommendations/blob/main/movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd


In [4]:
movies = pd.read_csv('dataset.csv')

In [5]:
movies.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

we have 9 columns but for our project we are not going to use all the columns because it is not useful to us. we can use some of the columns only.

For our project we have used id,title,genre and overeview because they are the most important for recommending movies.

we have done feature selection in the below parts

In [6]:
movies.columns


Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [7]:
movies = movies [['id','title','genre','overview']]
movies['tags'] = movies['overview'] + movies['genre']
new_data = movies.drop(columns = ['overview', 'genre'])
new_data

<ipython-input-7-8c721cd0f5f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['overview'] + movies['genre']


,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer


In [9]:
cv = CountVectorizer(max_features = 10000, stop_words = 'english')
cv



CountVectorizer(max_features=10000, stop_words='english')

In [10]:
vector =  cv.fit_transform(new_data['tags'].values.astype('U')).toarray()
vector.shape

(10000, 10000)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.07559289, 0.11065667,
        0.06388766],
       [0.05634362, 1.        , 0.07624929, ..., 0.        , 0.03636965,
        0.        ],
       [0.12888482, 0.07624929, 1.        , ..., 0.02273314, 0.06655583,
        0.08645856],
       ...,
       [0.07559289, 0.        , 0.02273314, ..., 1.        , 0.03253   ,
        0.02817181],
       [0.11065667, 0.03636965, 0.06655583, ..., 0.03253   , 1.        ,
        0.0412393 ],
       [0.06388766, 0.        , 0.08645856, ..., 0.02817181, 0.0412393 ,
        1.        ]])

In [16]:
new_data[new_data['title'] == "The Godfather"].index[0]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10000 non-null  int64 
 1   title   10000 non-null  object
 2   tags    9985 non-null   object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [26]:
distance = sorted(list(enumerate(similarity[2])), reverse = True, key= lambda vector: vector[1])
for i in distance[0:5]:
  print(new_data.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City


In [29]:
def recommend_movies(movies):
  index = new_data[new_data['title'] == movies ].index[0]
  distance = sorted(list(enumerate(similarity[index])), reverse = True, key= lambda vector: vector[1])
  for i in distance[0:5]:
    print(new_data.iloc[i[0]].title)


In [30]:
recommend_movies("Iron Man")

Iron Man
Iron Man 3
Guardians of the Galaxy Vol. 2
Avengers: Age of Ultron
Star Wars: Episode III - Revenge of the Sith


In [31]:
import pickle

In [32]:
pickle.dump(new_data, open('movies_list.pkl', 'wb'))

In [35]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [36]:
pickle.load(open('movies_list.pkl', 'rb'))

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...
